# Genetic Algorithm - 1D Example

# How to create arrays and apply math functions to them?

In [ ]:
import numpy as np

x = np.arange(0, np.pi, 0.2)
y = np.sin(x)
print(f"sine applied to the array: \n{x} \ngives: \n {y}")

## How to plot results?

In [ ]:
import matplotlib.pyplot as plt
from IPython import get_ipython

# For nicer animations in jupyter
get_ipython().run_line_magic("matplotlib", "widget")

x = np.arange(0, np.pi, 0.1)
y = np.sin(x)
plt.scatter(x, y)
plt.show()

## How do you write a function?

In [ ]:
def f(x):
    return x * x


print(f"square of 2: {f(2)}")
print(f"square of [1, 2, 3, 4]: {f(np.array([1,2,3,4]))}")

# Evolutionary algorithms
Evolutionary algorithms (EA) are population-based optimization algorithms used to find global optima. The algorithms are inspired from theory of evolution: a population evolves due to reproduction, mutation, and environmental selection ("Survival of the fittest" - Darwin's evolutionary theory).

- Problem: Optimize objective f(x) with input variables x.
- Solve using evolutionary algorithm:
    1. Generate an initial population of individuals (Here individual x encapsulates the input variables)
    2. Evaluate fitness and select a subset of fit individuals (Here fitness is related to objective to be optimized)
    3. Breed new individuals using crossover and mutation operators
    4. Repeat steps 2, 3 until a stopping criteria is fulfilled (e.g. negligible change in population)


Some surprising solutions obtained using EA: http://www.karlsims.com/evolved-virtual-creatures.html

# Genetic algorithms (GA)
- Popular among evolutionary algorithms.
- Each individual is represented as a binary string.
- Size of new generation is same as the old generation.
- Emphasis is on recombination operator rather than the mutation operator
- Probabilistic selection
- Weak selection with large population size (= 50 in general)

# Using GA, maximize the following objective functions.

In [ ]:
def f1(x):
    a1 = 950
    b1 = 10
    c1 = 2
    a2 = 200
    b2 = 25
    c2 = 2
    d = 10
    return (
        a1 * np.exp(-((x - b1) ** 2) / (2 * c1 ** 2))
        + a2 * np.exp(-((x - b2) ** 2) / (2 * c2 ** 2))
        + d
    )

def f2(x):
    return x ** 2 + 10

def f3(x):
    return (
        (-0.0004) * (x - 0) * (x - 7) * (x - 14) * (1 * x - 21) * (x - 28) * (x - 31)
        + 250.0
        + 18 * np.sin(7 * x)
    )


def f4(x):
    omega = 2.0
    sigma = 10.0
    c = 1000.0
    return (c * np.sin(omega * x) ** 2) * np.exp(-x / sigma)

# First plot the objective functions

In [ ]:
x = np.arange(0.0, 31.001, 0.001)

fig, ax = plt.subplots(figsize=(10, 5))
ax.set_xlim([0, 31])
ax.set_ylim([0, 1000])
ax.set_xlabel("x")
ax.set_ylabel("f(x)", rotation=0)
ax.plot(x, f1(x), label="f1(x)")
ax.plot(x, f2(x), label="f2(x)")
ax.plot(x, f3(x), label="f3(x)")
ax.plot(x, f4(x), label="f4(x)")
ax.legend(loc=0)
plt.show()

# Select an objective functions (f1 to f4) to maximize.
Which function is the most difficult to maximize?

In [ ]:
f = f1

# Mapping an integer to a binary string
Say we have a design variable (v) ranging from 1 to 6 is to be mapped to a 4 bit string. Then v=1 is mapped to '0000', v=6 is mapped to '1111'. Everything in between is mapped linearly.

In [ ]:
# conversion functions:
def bin_to_int(i):
    return int(i, 2)


def int_to_bin(i):
    return np.binary_repr(i)


def bin_to_float(b, lb=0, ub=31.0):
    """
    Converts a binary string to a floating point number between
    lower and upper bounds [lb, ub] using linear mapping
    """
    max_b = "1" * len(b)
    val = lb + (bin_to_int(b) / bin_to_int(max_b) * (ub - lb))
    return val

In [ ]:
# test conversions
print("Integer 3 in binary form:", int_to_bin(3))
print("Convert back to integer form:", bin_to_int("11"))

convert_strs = ["00", "01", "10", "11"]
print(
    "Float map of {}: {}".format(
        convert_strs, [np.round(bin_to_float(b), 2) for b in convert_strs]
    )
)

# Randomly generate initial population

In [ ]:
def initial_population(n, string_length=10):
    """
    Creates the initial population with 'n' individuals.
    Each length of each individual string is fixed.
    """
    # generation is a set of individuals
    generation = []
    for i in range(n):
        individual = ""
        for j in range(string_length):
            individual += str(np.random.choice([0, 1]))
        generation.append(individual)
    return generation

In [ ]:
G0 = initial_population(n=4, string_length=5)
print("Initial Population: ", G0)
for i in G0:
    b = bin_to_float(i, 0, 31)
    print(f"Binary string: {i} represents {b}")

In [ ]:
# Define fitness for proportional selection
fitness = lambda x: f(x)

# Proportional selection:
- weak selection pressure
- can only take positive values for objective function
- outliers deteriorate the scheme

In [ ]:
import matplotlib.cm as cm
import matplotlib.colors as colors


def roulette_wheel(G, ax):
    """
    Returns a roulette wheel for a given population 'G' according to the
    fitness value of the individuals, 'ax' is an Axes object on which the
    roulette wheel is plotted. Bigger the fitness value, bigger the share in the wheel
    """
    fracs = []
    wheel = []
    labels = []
    color_vals = [i for i in range(len(G))]
    norm = colors.Normalize(0, len(G))
    cmap = cm.get_cmap("jet")
    color = cmap(norm(color_vals))

    # calculate fitness of each individual
    Gx = [bin_to_float(i) for i in G]
    fGx = [fitness(i) for i in Gx]

    # find fitness fractions of each individual
    for i in range(len(G)):
        labels.append("Ind-" + str(i))
        fracs.append(float(fGx[i]) / float(sum(fGx)))

    # draw pie chart
    ax.pie(fracs, labels=labels, colors=color, normalize=True)

    # For each individual, accumulate fractions upto i
    # These are used to figure out where the roulette
    # wheel stops, i.e., select individuals
    for i in range(len(fracs)):
        if i == 0:
            wheel.append(fracs[i])
        else:
            wheel.append(wheel[i - 1] + fracs[i])

    return wheel

In [ ]:
# plot an example roulette wheel
fig, ax = plt.subplots()
G0 = initial_population(5)
for i in range(len(G0)):
    print("Fitness of individual {} is {}".format(i, fitness(bin_to_float(G0[i]))))
W0 = roulette_wheel(G0, ax)
print("Wheel (CDF): ", W0)

# Selection for mating operator

In [ ]:
def mating(G, W):
    """
    Returns a mating pool. For each individual in the population 'G', the
    roulette wheel 'W' is turned once from which the id of a mate is selected
    """
    mates = []
    for _ in G:
        # spin the roulette wheel and find where the pin stops
        stop_point = np.random.rand()
        for i in range(len(W)):
            if i == 0:
                if stop_point <= W[i] and stop_point > 0:
                    mate = i
            else:
                if stop_point <= W[i] and stop_point > W[i - 1]:
                    mate = i
        mates.append(mate)
    return mates

In [ ]:
# Select mates from a population
mates = mating(G0, W0)
print(f"Selected mates from the population {G0}: {mates}")

# Crossover operator, a type of recombination operator

In [ ]:
def crossover(G, M1, M2, probability=1.0, keep="random", print_all=False):
    """
    Performs one-point crossover for a population 'G', two parents are selected using two mating pools 'M1' and 'M2'.

    Args:
        G ([str]): Generation (a population of individuals)
        M1 ([int]): Mating pool 1
        M2 ([int]): Mating pool 2
        probability (float): If probability = 1.0, crossover is performed
            for every individual in the population, if probability = 0.0,
            crossover is not performed, i.e. the closer the value of the
            probability is to zero, crossover is less likely to happen,
            the closer to 1.0, crossover is most likely to occur.
        keep (str, optional): Only one offspring is kept to form the new
            generation, the 'keep' variable controls this decision, keep can
            be 1, 2 or 'random', 1 for keeping the first offspring, 2 the
            second, and 'random' to decide with a coin toss.
        print_all (bool, optional): Print process.

    Returns:
        [str]: New generation
    """
    new_generation = []

    assert keep in ["random", 1, 2]

    if print_all:
        print(
            "|-{1:->12}---{2:->12}---{3:->12}---{4:->12}-|".format("", "", "", "", "")
        )
        print(
            "| {1:>12}   {2:>12}   {3:>12}   {4:>12} |".format(
                "", "Crossover:", "", "", ""
            )
        )
        print(
            "|-{1:->12}---{2:->12}---{3:->12}---{4:->12}-|".format("", "", "", "", "")
        )
        print(
            "| {1:^12} | {2:^12} | {3:^12} | {4:^12} |".format(
                "", "Id", "Parents", "Child", "Keep"
            )
        )
        print(
            "|-{1:->12}---{2:->12}---{3:->12}---{4:->12}-|".format("", "", "", "", "")
        )

    for i in range(len(G)):
        # figure out where to crossover
        if np.random.rand() < probability:
            cp = np.random.randint(1, len(G[i]) - 1)  # cp .. cross point
        else:
            cp = 0
        parent_1 = G[M1[i]][:cp] + "|" + G[M1[i]][cp:]
        parent_2 = G[M2[i]][:cp] + "|" + G[M2[i]][cp:]
        child_1 = G[M1[i]][:cp] + "|" + G[M2[i]][cp:]
        child_2 = G[M2[i]][:cp] + "|" + G[M1[i]][cp:]
        children = [child_1, child_2]

        # remove the divider
        if keep == "random":
            keep = np.random.choice([1, 2])

        # decide which child to keep (no pun intended)
        kept = children[keep - 1].replace("|", "")

        new_generation.append(kept)
        if print_all:
            print(
                "| {1:>12} | {2:>12} | {3:>12} | {4:>12} |".format(
                    "", "Ind-" + str(M1[i]), parent_1, child_1, kept
                )
            )
            print(
                "| {1:>12} | {2:>12} | {3:>12} | {4:>12} |".format(
                    "", "Ind-" + str(M2[i]), parent_2, child_2, ""
                )
            )
            print(
                "|-{1:->12}---{2:->12}---{3:->12}---{4:->12}-|".format(
                    "", "", "", "", ""
                )
            )
    return new_generation

In [ ]:
# test Crossover operator
M1 = mating(G0, W0)
M2 = mating(G0, W0)
print("Population: ", G0)
print("Mating indices 1: ", M1)
print("Mating indices 2: ", M2)
G1 = crossover(G0, M1, M2, print_all=True, keep="random")

In [ ]:
def mutation(G, probability=0.001, print_all=False):
    """Performs mutation for every individual in the given population with some probability.

    Args:
        G ([str]): Generation
        probability (float, optional): Mutation probablity.
        print_all (bool, optional): Print process.

    Returns:
        [type]: [description]
    """
    new_generation = []
    if print_all:
        print("|-{0:->12}---{1:->12}---{2:->12}-|".format("", "", ""))
        print("| {0:>12}   {1:>12}   {2:>12} |".format("Mutation:", "", ""))
        print("|-{0:->12}---{1:->12}---{2:->12}-|".format("", "", ""))
        print("| {0:^12} | {1:^12} | {2:^12} |".format("Id", "Original", "New"))
        print("|-{0:->12}---{1:->12}---{2:->12}-|".format("", "", ""))

    for i, ind in enumerate(G):
        new_ind = ""
        for ch in ind:
            # with some probability, flip old value of each character
            if np.random.rand() < probability:
                if ch == "1":
                    new_ch = "0"
                if ch == "0":
                    new_ch = "1"
                new_ind += new_ch
            # Keep the old value
            else:
                new_ind += ch
        new_generation.append(new_ind)
        if print_all:
            print(
                "| {0:^12} | {1:^12} | {2:^12} |".format("Ind-" + str(i), ind, new_ind)
            )
    if print_all:
        print("|-{0:->12}---{1:->12}---{2:->12}-|".format("", "", ""))
    return new_generation

In [ ]:
print("Low probability of mutation:")
r1 = mutation(G0, print_all=True, probability=0.001)
print("High probability of mutation:")
r2 = mutation(G0, print_all=True, probability=1)

In [ ]:
# print fitness of a generation
def print_fitness(G):
    Gx = [bin_to_float(i) for i in G]
    fGx = [fitness(i) for i in Gx]
    print("|-{0:->12}---{1:->12}---{2:->12}---{3:->12}-|".format("", "", "", ""))
    print(
        "| {0:^12} | {1:^12} | {2:^12} | {3:^12} |".format(
            "Id", "Individual", "x Value", "Fitness"
        )
    )
    print("|-{0:->12}---{1:->12}---{2:->12}---{3:->12}-|".format("", "", "", ""))
    for i in range(len(G)):
        print(
            "| {0:^12} | {1:>12} | {2:>12.2f} | {3:12.2f} |".format(
                "Ind-" + str(i), G[i], Gx[i], fGx[i]
            )
        )
    print("|-{0:->12}---{1:->12}---{2:->12}---{3:->12}-|".format("", "", "", ""))
    print("| {0:<12}   {1:>12}   {2:>12} | {3:12.2f} |".format("Sum", "", "", sum(fGx)))
    print(
        "| {0:<12}   {1:>12}   {2:>12} | {3:12.2f} |".format(
            "Average", "", "", sum(fGx) / float(len(fGx))
        )
    )
    print("| {0:<12}   {1:>12}   {2:>12} | {3:12.2f} |".format("Max", "", "", max(fGx)))
    print("|-{0:->12}---{1:->12}---{2:->12}---{3:->12}-|".format("", "", "", ""))

In [ ]:
print_fitness(G0)

In [ ]:
import itertools


def plot_population(G, ax, alpha=1.0):
    """Plot individuals with different colors.

    Args:
        G ([str]): Population
        ax ([type]): Matplotlib axes
        alpha (float, optional): Transparency of each marker (individual).
    """
    # calculate fitness
    Gx = [bin_to_float(i) for i in G]
    fGx = [f(i) for i in Gx]

    # iterate over base colors
    color_iterator = itertools.cycle(colors.BASE_COLORS.keys())

    # scatter plot
    for i in range(len(Gx)):
        ax.scatter(Gx[i], fGx[i], c=next(color_iterator), s=70, alpha=alpha)

In [ ]:
# show fitness of the population
_, ax = plt.subplots()
plot_population(G0, ax)
ax.plot(x, f(x))
plt.show()

# GA algorithm:
1. Generate initial population
2. Evaluate fitness of population
3. Select individuals based on fitness for mutation selection (Note that there is no environmental selection)
4. Use recombination operator (Here crossover operator is used)
5. Use (weak) mutation operator
6. Repeat steps 2-5 until optimality criteria is fulfilled

In [ ]:
# initialize population and iteration count
import time

n = 20  # population size
G = initial_population(n)
t_max = 20  # number of iterations

# setting up plot layout
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))

In [ ]:
# For each iteration, plot the new generation
for t in range(t_max):
    # clear axis
    ax1.clear()
    ax2.clear()

    # set up axes layout
    ax1.grid()
    ax1.set_xlabel("x")
    ax1.set_ylabel("f(x)", rotation=0)
    ax1.set_xlim([0, 31])
    ax1.set_ylim([0, 1000])
    ax2.set_aspect("equal")

    # plot population
    fig.suptitle(f"Iteration = {t}")
    ax1.plot(x, f(x))
    plot_population(G, ax1, alpha=0.5)

    # prepare new generation
    W = roulette_wheel(G, ax2)
    M1 = mating(G, W)
    M2 = mating(G, W)
    C = crossover(G, M1, M2, probability=0.7, keep="random")
    G = mutation(C, probability=0.01)

    # output to figure
    fig.canvas.draw()
    fig.canvas.flush_events()
    time.sleep(1)

print_fitness(G)

In [ ]:
# Minimize the objective
fitness = lambda x: 1 / f(x)

In [ ]:
# Increases selective pressure
fitness = lambda x: f(x) ** 2

# Optional exercises:
- GA finds a global maximum if fitness-based selection schemes is used. How do we find a minimum?
- What does exponentional scaling of the objective function do?
- How do we keep the best individuals?
- Improve crossover operator: Use multi-point crossover and uniform crossover instead of one-point crossover
- Improve mutation operator: adaptively decrease the mutation probability
- Improve diversity of solutions found e.g. find all local optima
- Implement GA for a structural optimization problem